In [8]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import numpy as np

# Retrieve the data from rds database

* obtain the Nasdaq data, oil price data and Tweet news data.

In [9]:
conn = psycopg2.connect(
    database="example",
    user="postgres",
    password="password",
    host="database-2.cmuyi10lpj9v.us-west-1.rds.amazonaws.com",
    port='5432'
)
cur = conn.cursor()
cur.execute("""SELECT * FROM sentiments""")
rows = cur.fetchall()
sentiments = pd.DataFrame(rows)

cur.execute("""SELECT * FROM nasdaq""")
rows = cur.fetchall()
nasdaqDF = pd.DataFrame(rows)

cur.execute("""SELECT * FROM eta""")
rows = cur.fetchall()
etaDF = pd.DataFrame(rows)

conn.close()

In [10]:
# rename the for the oil table coulmns 
etaDF.rename(columns= { 0: 'id',1: 'date',2: 'price', 3: 'diff', 4: 'label'}, inplace= True) 
etaDF.head(3)

,id,date,price,diff,label
0,1,2006-06-12,70.28,-1.34,0
1,2,2006-06-13,68.48,-1.80,0
2,3,2006-06-14,69.12,0.64,1


In [11]:
# rename the for the tweet news table coulmns 
sentiments.rename (columns= { 0: 'id',1: 'date',2: 'Compound', 3: 'Positive', 4: 'Negative',5: 'Neutral'}, inplace= True) 
sentiments

,id,date,Compound,Positive,Negative,Neutral
0,0,2010-01-02,-0.201375,0.049500,0.752375,0.198125
1,1,2010-01-03,-0.001012,0.063750,0.865750,0.070500
2,2,2010-01-04,0.098568,0.094909,0.871636,0.033500
3,3,2010-01-05,0.010268,0.056636,0.912818,0.030545
4,4,2010-01-06,-0.025283,0.056500,0.890125,0.053375
...,...,...,...,...,...,...
3813,3813,2020-11-10,0.057316,0.072629,0.883677,0.043694
3814,3814,2020-11-11,0.049574,0.063011,0.903977,0.033011
3815,3815,2020-11-12,0.060377,0.076276,0.876759,0.046989
3816,3816,2020-11-13,0.030476,0.059390,0.873232,0.067390


In [12]:
# rename the for the Nasdaq table coulmns 
nasdaqDF = nasdaqDF.rename(columns={4:'label',1:'date'})
nasdaqDF.drop_duplicates(['date'], keep='last')
nasdaqDF

,0,date,2,3,label
0,1,2006-05-30,2164.74,-45.63,0
1,2,2006-05-31,2178.88,14.14,1
2,3,2006-06-01,2219.86,40.98,1
3,4,2006-06-02,2219.41,-0.45,0
4,5,2006-06-05,2169.62,-49.79,0
...,...,...,...,...,...
3637,2051,2014-07-14,4440.42,24.93,1
3638,2052,2014-07-15,4416.39,-24.03,0
3639,2053,2014-07-16,4425.97,9.58,1
3640,2054,2014-07-17,4363.45,-62.52,0


# Merge data 

* merge the Nasdaq and tweetnews table.

In [13]:
nasdaq_newsDF = pd.merge(nasdaqDF, sentiments, how="inner", on = ["date","date"])
nasdaq_newsDF = nasdaq_newsDF.dropna()
nasdaq_newsDF

,0,date,2,3,label,id,Compound,Positive,Negative,Neutral
0,2056,2014-07-21,4424.70,-7.45,0,1534,-0.140688,0.051231,0.827769,0.121000
1,2057,2014-07-22,4456.02,31.32,1,1535,-0.093103,0.059043,0.847043,0.093913
2,2058,2014-07-23,4473.70,17.68,1,1536,-0.060000,0.060761,0.846690,0.092549
3,2059,2014-07-24,4472.11,-1.59,0,1537,-0.162829,0.043673,0.828898,0.127449
4,2060,2014-07-25,4449.56,-22.55,0,1538,-0.012870,0.065132,0.864943,0.069943
...,...,...,...,...,...,...,...,...,...,...
2686,2051,2014-07-14,4440.42,24.93,1,1527,0.045877,0.071585,0.873569,0.054846
2687,2052,2014-07-15,4416.39,-24.03,0,1528,0.030637,0.070402,0.865439,0.064146
2688,2053,2014-07-16,4425.97,9.58,1,1529,-0.055856,0.055137,0.837192,0.107671
2689,2054,2014-07-17,4363.45,-62.52,0,1530,0.015665,0.064833,0.880222,0.054944


In [14]:
nasdaq_newsDF = nasdaq_newsDF.loc[:, ['label','date','Compound','Positive','Negative','Neutral']]
nasdaq_newsDF

,label,date,Compound,Positive,Negative,Neutral
0,0,2014-07-21,-0.140688,0.051231,0.827769,0.121000
1,1,2014-07-22,-0.093103,0.059043,0.847043,0.093913
2,1,2014-07-23,-0.060000,0.060761,0.846690,0.092549
3,0,2014-07-24,-0.162829,0.043673,0.828898,0.127449
4,0,2014-07-25,-0.012870,0.065132,0.864943,0.069943
...,...,...,...,...,...,...
2686,1,2014-07-14,0.045877,0.071585,0.873569,0.054846
2687,0,2014-07-15,0.030637,0.070402,0.865439,0.064146
2688,1,2014-07-16,-0.055856,0.055137,0.837192,0.107671
2689,0,2014-07-17,0.015665,0.064833,0.880222,0.054944


# Pre-processing

* Preprocess the dataset prior to fitting the model.
* Perform feature selection and remove unnecessary features.
* Use `MinMaxScaler` to scale the numerical data.
* Separate the data into training and testing data.

In [15]:
y = nasdaq_newsDF["label"]
X = nasdaq_newsDF.loc[:, ['Compound','Positive','Negative','Neutral']]
print(X.shape, y.shape)

(2691, 4) (2691,)


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

C:\Users\emmax\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\emmax\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\emmax\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\emmax\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [18]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Train the Support Vector Machine